In [1]:
%pip install apyori

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [3]:
dataset = pd.read_csv("../datasets/Market_Basket_Optimisation.csv", header=None)
print(len(dataset))
transactions = []
for transaction in dataset.values:
    transactions.append([product for product in transaction if isinstance(product, str)])

0

7501


0

In [4]:
from apyori import apriori

daily_recurrence_threshold = 3
min_support = 7 * daily_recurrence_threshold / len(transactions)
min_confidence = 0.2 # set based on experimentation and checking how many rules we're getting  
association_product_count = 2
rules = apriori(transactions=transactions,
                min_support=min_support,
                min_confidence=min_confidence,
                min_lift=3,
                min_length=association_product_count,
                max_length=association_product_count
                )

rules_as_list = list(rules)

0

0

In [10]:
def relations_with_higher_confidence(rules: list) -> np.array:
    """
    returns: list<[idx, base_item, added_item, lift, confidence, support]>
    """
    results = []
    for idx_to_rule in enumerate(rules):
        rule = idx_to_rule[1]
        ordered_stat_list = rule.ordered_statistics
        max_stat = max(ordered_stat_list, key=lambda i: i.confidence)
        results.append([idx_to_rule[0], list(max_stat.items_base)[0], list(max_stat.items_add)[0], max_stat.lift, max_stat.confidence, rule.support])
    result_as_numpy_array = np.array(results)[:, 1:]
    return result_as_numpy_array


relations_with_higher_confidence = relations_with_higher_confidence(rules_as_list)
results_data_frame = pd.DataFrame(relations_with_higher_confidence, columns=['Base item', 'Added item', 'Lift', 'Confidence', 'Support'])
desc_by_lift = results_data_frame.sort_values('Lift', ascending=False)
desc_by_lift

,Base item,Added item,Lift,Confidence,Support
4,fromage blanc,honey,5.164270764485569,0.2450980392156863,0.003332888948140248
1,light cream,chicken,4.84395061728395,0.29059829059829057,0.004532728969470737
3,pasta,escalope,4.700811850163794,0.3728813559322034,0.005865884548726837
9,pasta,shrimp,4.506672147735896,0.3220338983050847,0.005065991201173177
8,whole wheat pasta,olive oil,4.122410097642296,0.2714932126696833,0.007998933475536596
0,extra dark chocolate,chicken,3.8894074074074076,0.23333333333333334,0.0027996267164378083
6,tomato sauce,ground beef,3.840659481324083,0.3773584905660377,0.005332622317024397
2,mushroom cream sauce,escalope,3.790832696715049,0.3006993006993007,0.005732568990801226
5,herb & pepper,ground beef,3.2919938411349285,0.3234501347708895,0.015997866951073192
7,light cream,olive oil,3.1147098515519573,0.20512820512820515,0.003199573390214638
